In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta


In [18]:
def validate_dataset(df): #check if the dataset is empty
    if df.empty:
        return False
    return True
def fill_missing_values(df): #fill missing values with 1
    df['Size'].fillna(1, inplace=True)
def date_sort(df):      #sort the dataset by date
    df['Date'] = pd.to_datetime(df['Date'])
    df_sorted = df.sort_values(by='Date')
    df_sorted.reset_index(drop=True, inplace=True)
    return df_sorted
def total_profit(df): #calculate the total profit
    win=0
    lose=0
    current_holding={} #store the current holding
    total_invested=0
    profit=0
    for idx, row in df.iterrows():
        
        if row['Symbol'] not in current_holding: #if the symbol is occuring for the first time
            if row['Side']=='buy':  #if the transaction is buy
                current_holding[row['Symbol']]=[row['Price'],row['Size']] #store the price and size of the share
            else:
                current_holding[row['Symbol']]=[row['Price'],-row['Size']] 
            total_invested+=row['Price']*row['Size']
        else: #if the symbol is already present
            if row['Side']=='buy':
                if current_holding[row['Symbol']][1]>0:
                    current_holding[row['Symbol']][0]=(row['Price']*row['Size']+current_holding[row['Symbol']][0]*
                                                       current_holding[row['Symbol']][1])/(current_holding[row['Symbol']][1]+row['Size']) # weighted average of the price
                    current_holding[row['Symbol']][1]+=row['Size']
                    
                else:
                    if current_holding[row['Symbol']][0]>row['Price']:
                        profit+=(current_holding[row['Symbol']][0]-row['Price'])*min(row['Size'],abs(current_holding[row['Symbol']][1])) #profit incremented
                        win+=1
                    else:
                        profit+=(current_holding[row['Symbol']][0]-row['Price'])*min(row['Size'],abs(current_holding[row['Symbol']][1]))
                        lose+=1
                    current_holding[row['Symbol']][1]+=row['Size']
                    if current_holding[row['Symbol']][1]>0:
                        current_holding[row['Symbol']][0]=row['Price']
                    
                        
                    
                    
            else:
                if current_holding[row['Symbol']][1]<0:
                    current_holding[row['Symbol']][0]=(row['Price']*row['Size']+current_holding[row['Symbol']][0]*
                                                       abs(current_holding[row['Symbol']][1]))/(abs(current_holding[row['Symbol']][1])+row['Size']) #weighted average of the price
                    current_holding[row['Symbol']][1]-=row['Size']
                    total_invested+=row['Price']*row['Size']
                else:
                    if current_holding[row['Symbol']][0]<row['Price']:
                        profit+=(row['Price']-current_holding[row['Symbol']][0])*min(row['Size'],abs(current_holding[row['Symbol']][1])) #profit incremented
                        win+=1
                    else:
                        profit+=(row['Price']-current_holding[row['Symbol']][0])*min(row['Size'],abs(current_holding[row['Symbol']][1]))
                        lose+=1

                    current_holding[row['Symbol']][1]-=row['Size']
                    if current_holding[row['Symbol']][1]<0:
                        current_holding[row['Symbol']][0]=row['Price']
        
        if current_holding[row['Symbol']][1]==0: # if the size of the holding is 0, delete the holding
            del current_holding[row['Symbol']]
    
    return profit
def drawdown_cal(profit_array):
    temp=0
    drawdown=0
    for i in range (len(profit_array)):
        if temp+profit_array[i]<0: 
            temp+=profit_array[i] #if the profit is negative, add it to the temporary variable
        else:
            temp=0
        drawdown=min(drawdown,temp)
    return drawdown
       
                   
def win_rate(df):
    win=0
    lose=0
    current_holding={}
    total_invested=0
    for idx, row in df.iterrows():
        
        if row['Symbol'] not in current_holding:
            if row['Side']=='buy':
                current_holding[row['Symbol']]=[row['Price'],row['Size']]
            else:
                current_holding[row['Symbol']]=[row['Price'],-row['Size']]
            total_invested+=row['Price']*row['Size']
        else:
            if row['Side']=='buy':
                if current_holding[row['Symbol']][1]>0:
                    current_holding[row['Symbol']][0]=(row['Price']*row['Size']+current_holding[row['Symbol']][0]*current_holding[row['Symbol']][1])/(current_holding[row['Symbol']][1]+row['Size'])
                    current_holding[row['Symbol']][1]+=row['Size']
                    
                else:
                    if current_holding[row['Symbol']][0]>row['Price']:
                        win+=1 #increment win in case of profit
                    else:
                        lose+=1 # increment loss in case of loss
                    current_holding[row['Symbol']][1]+=row['Size']
                    if current_holding[row['Symbol']][1]>0:
                        current_holding[row['Symbol']][0]=row['Price']
                    
                        
                    
                    
            else:
                if current_holding[row['Symbol']][1]<0:
                    current_holding[row['Symbol']][0]=(row['Price']*row['Size']+current_holding[row['Symbol']][0]*abs(current_holding[row['Symbol']][1]))/(abs(current_holding[row['Symbol']][1])+row['Size'])
                    current_holding[row['Symbol']][1]-=row['Size']
                    total_invested+=row['Price']*row['Size']
                else:
                    if current_holding[row['Symbol']][0]<row['Price']:
                        win+=1 #increment win in case of profit
                    else:
                        lose+=1    # increment lose in case of loss

                    current_holding[row['Symbol']][1]-=row['Size']
                    if current_holding[row['Symbol']][1]<0:
                        current_holding[row['Symbol']][0]=row['Price']
        if current_holding[row['Symbol']][1]==0:
            del current_holding[row['Symbol']]
    return win/(win+lose)
def total_trades(df):
    return len(df)
def total_long_trades(df):
    return len(df[df['Side']=='buy']) #count the number of buy transactions
def total_short_trades(df):
    return len(df[df['Side']=='sell'])#count the number of sell transactions
def average_return_per_trade(df): #same as total profit
    win=0
    lose=0
    current_holding={}
    total_invested=0
    profit=0
    for idx, row in df.iterrows():
        
        
        if row['Symbol'] not in current_holding:
            if row['Side']=='buy':
                current_holding[row['Symbol']]=[row['Price'],row['Size']]
            else:
                current_holding[row['Symbol']]=[row['Price'],-row['Size']]
            total_invested+=row['Price']*row['Size']
        else:
            if row['Side']=='buy':
                if current_holding[row['Symbol']][1]>0:
                    current_holding[row['Symbol']][0]=(row['Price']*row['Size']+current_holding[row['Symbol']][0]*current_holding[row['Symbol']][1])/(current_holding[row['Symbol']][1]+row['Size'])
                    current_holding[row['Symbol']][1]+=row['Size']
                    
                else:
                    if current_holding[row['Symbol']][0]>row['Price']:
                        profit+=(current_holding[row['Symbol']][0]-row['Price'])*min(row['Size'],abs(current_holding[row['Symbol']][1]))
                        win+=1
                    else:
                        profit+=(current_holding[row['Symbol']][0]-row['Price'])*min(row['Size'],abs(current_holding[row['Symbol']][1]))
                        lose+=1
                    current_holding[row['Symbol']][1]+=row['Size']
                    if current_holding[row['Symbol']][1]>0:
                        current_holding[row['Symbol']][0]=row['Price']
                    
                        
                    
                    
            else:
                if current_holding[row['Symbol']][1]<0:
                    current_holding[row['Symbol']][0]=(row['Price']*row['Size']+abs(current_holding[row['Symbol']][0])*current_holding[row['Symbol']][1])/(abs(current_holding[row['Symbol']][1])+row['Size'])
                    current_holding[row['Symbol']][1]-=row['Size']
                    total_invested+=row['Price']*row['Size']
                else:
                    if current_holding[row['Symbol']][0]<row['Price']:
                        profit+=(row['Price']-current_holding[row['Symbol']][0])*min(row['Size'],abs(current_holding[row['Symbol']][1]))
                        win+=1
                    else:
                        profit+=(row['Price']-current_holding[row['Symbol']][0])*min(row['Size'],abs(current_holding[row['Symbol']][1]))
                        lose+=1

                    current_holding[row['Symbol']][1]-=row['Size']
                    if current_holding[row['Symbol']][1]<0:
                        current_holding[row['Symbol']][0]=row['Price']
        
        if current_holding[row['Symbol']][1]==0:
            del current_holding[row['Symbol']]
     
    return profit/(win+lose)


def load_data(url):
    df = pd.read_csv(url)
    return df
def column_removal(df):
    df=df.drop(columns=['disclosureYear','disclosureDate','owner','district','representative','capitalGainsOver200USD','option_symbol','assetDescription'])
    return df
def average_holding_period(df):
    win=0
    lose=0
    current_holding={}
    total_invested=0
    date_store=[]
    for idx, row in df.iterrows():
        
        if row['Symbol'] not in current_holding:
            current_holding[row['Symbol']]=[] #initiating queue for each symbol
            
            if row['Side']=='buy':
                current_holding[row['Symbol']].append([row['Date'],row['Size']]) #appending the date and size of the share
            else:
                current_holding[row['Symbol']].append([row['Date'],-row['Size']]) #negative for shorts
        else:
            if row['Side']=='buy':
                
                if current_holding[row['Symbol']][0][1]>0: #if the row side and holding side match append in the holding
                    current_holding[row['Symbol']].append([row['Date'],row['Size']])
                    
                    
                else:
                    size=row['Size']
                    
                    while(size>0): #decrement the current holding until either current size is 0 or holding size is 0
                        if len(current_holding[row['Symbol']])==0:
                                if size!=0:
                                    current_holding[row['Symbol']].append([row['Date'],size]) #indicated change in holding for symbol from sell to buy
                                    break
                                else:
                                    break

                        if abs(current_holding[row['Symbol']][0][1])>size:
                            current_holding[row['Symbol']][0][1]+=size
                            temp_date=row['Date']-current_holding[row['Symbol']][0][0]
                            date_store.append(temp_date.days) #append the difference in days
                            size=0 #loop breaks when size is 0
                            break
                        else:
                            temp_date=row['Date']-current_holding[row['Symbol']][0][0]
                            date_store.append(temp_date.days)
                            size+=current_holding[row['Symbol']][0][1]
                            current_holding[row['Symbol']].pop(0) #removing the top entry as size is 0
                
                        
                    
                    
            else:
                if current_holding[row['Symbol']][0][1]<0:
                    current_holding[row['Symbol']].append([row['Date'],-row['Size']])
                    
                else:
                    size=row['Size']
                    while(size>0):
                        if len(current_holding[row['Symbol']])==0:
                                if size!=0:
                                    current_holding[row['Symbol']].append([row['Date'],-size])
                                    break
                                else:
                                    break
                        if abs(current_holding[row['Symbol']][0][1])>size:
                            current_holding[row['Symbol']][0][1]-=size
                            temp_date=row['Date']-current_holding[row['Symbol']][0][0]
                            date_store.append(temp_date.days)
                            size=0
                            break
                        else:
                            temp_date=row['Date']-current_holding[row['Symbol']][0][0]
                            date_store.append(temp_date.days)
                            size-=current_holding[row['Symbol']][0][1]
                            current_holding[row['Symbol']].pop(0)
        
        if len(current_holding[row['Symbol']])==0:
            del current_holding[row['Symbol']]
    date_store=np.array(date_store)
    return np.mean(date_store)
def capital_invested(df): # calculation same as profit
    win=0
    lose=0
    current_holding={}
    total_invested=0
    for idx, row in df.iterrows():
        
        if row['Symbol'] not in current_holding:
            if row['Side']=='buy':
                current_holding[row['Symbol']]=[row['Price'],row['Size']]
            else:
                current_holding[row['Symbol']]=[row['Price'],-row['Size']]
        else:
            if row['Side']=='buy':
                if current_holding[row['Symbol']][1]>0:
                    current_holding[row['Symbol']][0]=(row['Price']*row['Size']+current_holding[row['Symbol']][0]*current_holding[row['Symbol']][1])/(current_holding[row['Symbol']][1]+row['Size'])
                    current_holding[row['Symbol']][1]+=row['Size']
                    
                else:
                    if current_holding[row['Symbol']][0]>row['Price']:
                        win+=1
                    else:
                        lose+=1
                    current_holding[row['Symbol']][1]+=row['Size']
                    if current_holding[row['Symbol']][1]>0:
                        current_holding[row['Symbol']][0]=row['Price']
                    
                        
                    
                    
            else:
                if current_holding[row['Symbol']][1]<0:
                    current_holding[row['Symbol']][0]=(row['Price']*row['Size']+abs(current_holding[row['Symbol']][0])*current_holding[row['Symbol']][1])/(abs(current_holding[row['Symbol']][1])+row['Size'])
                    current_holding[row['Symbol']][1]-=row['Size']
                    total_invested+=row['Price']*row['Size']
                else:
                    if current_holding[row['Symbol']][0]<row['Price']:
                        win+=1
                    else:
                        lose+=1

                    current_holding[row['Symbol']][1]-=row['Size']
                    if current_holding[row['Symbol']][1]<0:
                        current_holding[row['Symbol']][0]=row['Price']
            
        if current_holding[row['Symbol']][1]==0:
            del current_holding[row['Symbol']]
        temp_investment=0
        for key in current_holding: # calculates the total invested amount assuming short sells require capital equal to price of share as collateral
            temp_investment+=current_holding[key][0]*abs(current_holding[key][1])
        total_invested=max(total_invested,temp_investment)
    return total_invested
def standard_deviation(df):
    win=0
    lose=0
    current_holding={}
    total_invested=0
    profit=0
    profit_array=[]
    for idx, row in df.iterrows():
        
        
        if row['Symbol'] not in current_holding:
            if row['Side']=='buy':
                current_holding[row['Symbol']]=[row['Price'],row['Size']]
            else:
                current_holding[row['Symbol']]=[row['Price'],-row['Size']]
            total_invested+=row['Price']*row['Size']
        else:
            if row['Side']=='buy':
                if current_holding[row['Symbol']][1]>0:
                    current_holding[row['Symbol']][0]=(row['Price']*row['Size']+current_holding[row['Symbol']][0]*current_holding[row['Symbol']][1])/(current_holding[row['Symbol']][1]+row['Size'])
                    current_holding[row['Symbol']][1]+=row['Size']
                    
                else:
                    if current_holding[row['Symbol']][0]>row['Price']:
                        profit+=(current_holding[row['Symbol']][0]-row['Price'])*min(row['Size'],abs(current_holding[row['Symbol']][1]))
                        win+=1
                        profit_array.append((current_holding[row['Symbol']][0]-row['Price'])*min(row['Size'],abs(current_holding[row['Symbol']][1])))
                    else:
                        profit+=(current_holding[row['Symbol']][0]-row['Price'])*min(row['Size'],abs(current_holding[row['Symbol']][1]))
                        profit_array.append((current_holding[row['Symbol']][0]-row['Price'])*min(row['Size'],abs(current_holding[row['Symbol']][1])))
                        lose+=1
                    current_holding[row['Symbol']][1]+=row['Size']
                    if current_holding[row['Symbol']][1]>0:
                        current_holding[row['Symbol']][0]=row['Price']
                    
                        
                    
                    
            else:
                if current_holding[row['Symbol']][1]<0:
                    current_holding[row['Symbol']][0]=((row['Price']*row['Size']+abs(current_holding[row['Symbol']][0]))*current_holding[row['Symbol']][1])/(abs(current_holding[row['Symbol']][1])+row['Size'])
                    current_holding[row['Symbol']][1]-=row['Size']
                    total_invested+=row['Price']*row['Size']
                else:
                    if current_holding[row['Symbol']][0]<row['Price']:
                        profit+=(row['Price']-current_holding[row['Symbol']][0])*min(row['Size'],abs(current_holding[row['Symbol']][1]))
                        profit_array.append((row['Price']-current_holding[row['Symbol']][0])*min(row['Size'],abs(current_holding[row['Symbol']][1])))
                        win+=1
                    else:
                        profit+=(row['Price']-current_holding[row['Symbol']][0])*min(row['Size'],abs(current_holding[row['Symbol']][1]))
                        profit_array.append((row['Price']-current_holding[row['Symbol']][0])*min(row['Size'],abs(current_holding[row['Symbol']][1])))
                        lose+=1

                    current_holding[row['Symbol']][1]-=row['Size']
                    if current_holding[row['Symbol']][1]<0:
                        current_holding[row['Symbol']][0]=row['Price']
        
        if current_holding[row['Symbol']][1]==0:
            del current_holding[row['Symbol']]
    temp=profit_array
    profit_array=np.array(profit_array)

    return np.std(profit_array),temp
    
def sharpe_ratio(total_invested,profit,standard_deviation,risk_free_rate):
    return (profit/total_invested*100-risk_free_rate)/standard_deviation
def rename_columns(df):
    df.rename(columns={'transactionDate':'Date','ticker':'Symbol','amount':'Price','type':'Side'},inplace=True)
    return df
def convert_entry(entry): #convert the Side column to buy and sell
    if entry.startswith('Purchase'):
        return 'buy'
    elif entry.startswith('Sale'):
        return 'sell'
    return entry
def preprocessing(df): # preprocessing the dataset
    df['Size']=1
    df=column_removal(df)
    df=rename_columns(df)
    df['Side'] = df['Side'].apply(convert_entry)
    df['Price'] = df['Price'].str.extract(r'\$(\d{1,3}(?:,\d{3})*)', expand=False)
    df['Price'] = df['Price'].str.replace(',', '').astype(int)
    df['Date'] = pd.to_datetime(df['Date'])
    
    return df
    




    

    


                
        
        
        

    

    
                


        

    

            
                    
                
            




**Q1 RESULTS**


In [19]:
df=load_data("../data/raw/temp_data_backup.csv")
df=date_sort(df)
profit=total_profit(df)
stdev,profit_array=standard_deviation(df)
drawdown=drawdown_cal(profit_array)
total_trade=total_trades(df)
total_long_trade=total_long_trades(df)
total_short_trade=total_short_trades(df)
win_ratio=win_rate(df)
avg_return=average_return_per_trade(df)
avg_hold_period=average_holding_period(df)
stdev,profit_array=standard_deviation(df)
temp=capital_invested(df)
roi=profit/temp*100
risk_free_rate=7
sharpe_ratio_store= sharpe_ratio(temp,profit,stdev,risk_free_rate)
results=pd.Series([profit,drawdown,total_trade,total_long_trade,total_short_trade,win_ratio,avg_return,avg_hold_period,temp,roi,stdev,sharpe_ratio_store],index=['profit','drawdown','total_trade','total_long_trade','total_short_trade','win_ratio','avg_return','avg_hold_period','capital_invested','roi','stdev','sharpe_ratio'])
print(results)

profit               1400.000000
drawdown             -100.000000
total_trade             9.000000
total_long_trade        5.000000
total_short_trade       4.000000
win_ratio               0.800000
avg_return            280.000000
avg_hold_period         1.333333
capital_invested     1500.000000
roi                    93.333333
stdev                 312.409987
sharpe_ratio            0.276346
dtype: float64


**Q2 Results**

In [20]:
df=load_data("../data/raw/testData.csv")
df=preprocessing(df)
df=date_sort(df)
df.to_csv("../data/processed/testData_1.csv")
profit=total_profit(df)
stdev,profit_array=standard_deviation(df)
drawdown=drawdown_cal(profit_array)
total_trade=total_trades(df)
total_long_trade=total_long_trades(df)
total_short_trade=total_short_trades(df)
win_ratio=win_rate(df)
avg_return=average_return_per_trade(df)
avg_hold_period=average_holding_period(df)
stdev,profit_array=standard_deviation(df)
temp=capital_invested(df)
roi=profit/temp*100
risk_free_rate=7
sharpe_ratio_store= sharpe_ratio(temp,profit,stdev,risk_free_rate)
results=pd.Series([profit,drawdown,total_trade,total_long_trade,total_short_trade,win_ratio,avg_return,avg_hold_period,temp,roi,stdev,sharpe_ratio_store] ,index=['profit','drawdown','total_trade','total_long_trade','total_short_trade','win_ratio','avg_return','avg_hold_period','capital_invested','roi','stdev','sharpe_ratio'])
print(results)




profit               1.204265e+06
drawdown            -4.821335e+06
total_trade          1.020000e+02
total_long_trade     4.000000e+01
total_short_trade    6.200000e+01
win_ratio            6.388889e-01
avg_return          -2.424622e+04
avg_hold_period      1.463611e+02
capital_invested     3.298045e+06
roi                  3.651451e+01
stdev                2.288380e+05
sharpe_ratio         1.289756e-04
dtype: float64
